In [1]:
import pandas as pd
import numpy as np

from IPython.display import display

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
pd.set_option('display.width', 15000)
pd.set_option('display.max_columns', 100)
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_context('poster')
%matplotlib inline

In [2]:
# Load data

data_dir = 'data/Champaign/'
dfb = pd.read_csv(data_dir + 'business.csv', encoding='latin-1')
dfr = pd.read_csv(data_dir + 'review.csv', encoding='latin-1')
dfu = pd.read_csv(data_dir + 'user.csv', encoding='latin-1')
datar = pd.read_csv(data_dir + 'data_review.csv', encoding='latin-1')

print(dfb.shape)
display(dfb.head())
print(dfr.shape)
display(dfr.head())
print(dfu.shape)
display(dfu.head())
print(datar.shape)
display(datar.head())

(1084, 101)


,attributes.Ambience.divey,attributes.RestaurantsDelivery,attributes.DogsAllowed,postal_code,hours.Thursday,attributes.HairSpecializesIn.coloring,attributes.BestNights.sunday,attributes.BYOB,attributes.AgesAllowed,attributes.Music.video,hours.Friday,latitude,attributes.Alcohol,attributes.Ambience.classy,attributes.RestaurantsTableService,business_id,attributes.Ambience.touristy,attributes.RestaurantsCounterService,attributes.Corkage,attributes.RestaurantsGoodForGroups,categories,name,attributes.BusinessAcceptsBitcoin,attributes.HappyHour,attributes.WheelchairAccessible,attributes.Ambience.hipster,attributes.BusinessAcceptsCreditCards,is_open,attributes.DietaryRestrictions.vegetarian,attributes.Music.live,attributes.Music.background_music,neighborhood,attributes.BusinessParking.lot,attributes.Music.karaoke,review_count,attributes.GoodForMeal.breakfast,attributes.NoiseLevel,attributes.HairSpecializesIn.perms,state,attributes.DriveThru,attributes.HasTV,attributes.GoodForMeal.dinner,attributes.BusinessParking.street,address,attributes.RestaurantsAttire,hours.Sunday,attributes.BestNights.tuesday,attributes.AcceptsInsurance,attributes.BestNights.wednesday,hours.Wednesday,...,attributes.Open24Hours,attributes.Ambience.trendy,attributes.CoatCheck,hours.Monday,attributes.HairSpecializesIn.straightperms,city,attributes.HairSpecializesIn.curly,attributes.Music.no_music,hours.Tuesday,attributes.HairSpecializesIn.africanamerican,stars,attributes.RestaurantsPriceRange2,attributes.Ambience.intimate,attributes.GoodForMeal.latenight,attributes.GoodForMeal.dessert,attributes.BusinessParking.validated,attributes.GoodForMeal.lunch,attributes.GoodForKids,attributes.DietaryRestrictions.soy-free,attributes.GoodForMeal.brunch,attributes.BusinessParking.valet,longitude,attributes.DietaryRestrictions.gluten-free,attributes.BYOBCorkage,attributes.BusinessParking.garage,attributes.BestNights.friday,hours.Saturday,attributes.Music.dj,attributes.HairSpecializesIn.extensions,attributes.BestNights.saturday,attributes.Ambience.casual,attributes.BestNights.thursday,attributes.BestNights.monday,attributes.HairSpecializesIn.asian,attributes.DietaryRestrictions.kosher,attributes.WiFi,attributes.Smoking,attributes.DietaryRestrictions.halal,attributes.GoodForDancing,attributes.ByAppointmentOnly,attributes.Caters,attributes.RestaurantsReservations,attributes.DietaryRestrictions.dairy-free,attributes.DietaryRestrictions.vegan,attributes.Ambience.romantic,attributes.Music.jukebox,attributes.Ambience.upscale,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,NaN,NaN,NaN,61820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.094068,NaN,NaN,NaN,56,NaN,NaN,NaN,True,"[u'Restaurants', u'Mexican']",Fiesta Ranchera,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,4,False,quiet,NaN,IL,NaN,NaN,False,NaN,1805 S Neil St,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Champaign,NaN,NaN,NaN,NaN,2.0,NaN,NaN,False,False,NaN,False,NaN,NaN,False,NaN,-88.245785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,61820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.112407,NaN,NaN,NaN,57,NaN,NaN,NaN,NaN,"[u'Property Management', u'Real Estate', u'Apa...",Professional Property Management,NaN,NaN,NaN,NaN,False,1,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,IL,NaN,NaN,NaN,NaN,401 E Springfield Ave,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Champaign,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-88.233285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,False,NaN,61820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.137569,none,False,NaN,259,False,NaN,NaN,True,"[u'Fast Food', u'Restaurants', u'Burgers']",McDonald's,NaN,NaN,NaN,False,True,1,NaN,NaN,NaN,NaN,False,NaN,10,False,NaN,NaN,IL,True,True,False,False,1812 N Neil St,casual,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,NaN,Champaign,NaN,NaN,NaN,NaN,1.5,1.0,False,False,False,False,False,True,NaN,False,F

(24088, 9)


,business_id,funny,user_id,review_id,text,stars,date,useful,cool
0,56,0,94431,8Zd_i4Kwb1uzzvrAU37_9A,Going here really boils down to a question of ...,2,2012-05-13,0,0
1,56,0,397042,fG8WAIDl4GZQYRYEfzNH6g,Me and my girlfriend walked into this place af...,1,2012-03-08,0,0
2,56,1,259145,cU39f51OCGzn5vF_TvtO-g,I used to really like El Toro which was in the...,3,2011-04-11,4,2
3,56,0,412466,1lRmEQYCMtgG-bdPGtgudQ,"As others have said, steer clear of this place...",1,2011-06-20,1,0
4,57,0,84515,iX93LIaKvOvH152-5HeIQQ,I stayed at one of their Urbana location for o...,4,2014-08-16,0,0


(9142, 22)


,user_id,yelping_since,useful,compliment_photos,compliment_list,compliment_funny,compliment_plain,review_count,elite,fans,compliment_note,funny,compliment_writer,compliment_cute,average_stars,compliment_more,friends,compliment_hot,cool,name,compliment_profile,compliment_cool
0,94431,2010-05-14,4,0,0,0,0,25,[],1,1,1,0,0,3.08,0,"[u'b2oXgL-sQJn2w8pw6wKCSA', u'7ZFy67BWtPQsptv-...",1,1,Rick,0,0
1,397042,2010-09-11,26,0,0,0,0,2,[],0,0,0,0,0,1.00,0,[],0,1,Damian,0,0
2,259145,2010-10-17,13541,77,24,953,775,1097,"[2015, 2011, 2013, 2012, 2014, 2016, 2010, 2017]",88,387,7463,531,50,4.30,68,"[u'_4fa15cUZDxtLE7pMkw4kw', u'XeQu8M47hx7l1cUX...",537,10077,Kathleen,48,953
3,412466,2009-07-06,374,0,0,4,12,220,"[2015, 2014, 2017, 2016]",9,5,43,4,0,4.07,0,"[u'kGXk_5X6wTHZsYlUDJLeIg', u'KjZCO2viXOF_xcpT...",1,22,Nick,1,4
4,84515,2014-08-13,63,0,0,0,1,189,[],3,0,11,0,0,3.84,0,"[u'Azi0eHGZaVIZCgDuCzRlGw', u'7bmSgEJk3B4Bkt0E...",0,4,Francisco,0,0


(24088, 3)


,user_id,business_id,stars
0,94431,56,2
1,397042,56,1
2,259145,56,3
3,412466,56,1
4,84515,57,4
